# Conformación dataset

In [ ]:
import glob
import xarray as xr
import geopandas as gpd

# Ruta al archivo shape
shapefile_path = "../shapes/shape_boyaca.shp"

# Carpeta que contiene los archivos .nc (Descargando usando el notebook 1.get_data.ipynb)
nc_folder_path = "../data/archivos_climate_serv/"

# Cargar el archivo shape
gdf = gpd.read_file(shapefile_path)

# Obtener las coordenadas extremas iniciales
min_latitude = float('inf')
max_latitude = float('-inf')
min_longitude = float('inf')
max_longitude = float('-inf')

# Iterar sobre los polígonos del archivo shape
for i, geometry in enumerate(gdf.geometry):
    # Obtener los límites de latitud y longitud del polígono actual
    polygon_bounds = geometry.bounds
    
    # Actualizar los valores máximos y mínimos de latitud y longitud
    min_latitude = min(min_latitude, polygon_bounds[1])
    max_latitude = max(max_latitude, polygon_bounds[3])
    min_longitude = min(min_longitude, polygon_bounds[0])
    max_longitude = max(max_longitude, polygon_bounds[2])

min_longitude -= 0.001
max_longitude += 0.001

# Crear una lista para almacenar los datos filtrados
filtered_datasets = []

# Obtener la lista de archivos .nc en la carpeta
nc_file_paths = glob.glob(nc_folder_path + "*.nc")

# Iterar sobre los archivos .nc
for nc_file_path in nc_file_paths:
    # Leer el archivo .nc utilizando xarray
    ds = xr.open_dataset(nc_file_path)
    
    # Filtrar por latitud y longitud máximas y mínimas
    ds_filtered = ds.sel(
        longitude=slice(min_longitude, max_longitude),
        latitude=slice(min_latitude, max_latitude)
    )
    
    # Agregar el dataset filtrado a la lista
    filtered_datasets.append(ds_filtered)

# Combinar los datasets filtrados en uno solo
combined_dataset = xr.concat(filtered_datasets, dim="time")

# Ruta para guardar el archivo .nc combinado
output_nc_path = "../data/data_Boyaca.nc"

# Guardar el dataset combinado en un nuevo archivo .nc
combined_dataset.to_netcdf(output_nc_path)